In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xl
import re
import itertools
from datetime import datetime
pd.options.mode.chained_assignment = None



## TODO create a front end interface to select which tool and contest date
tool = "DK"
# tool = "Yahoo"
sport = "NHL"
# sport = "NBA"
contest_date = "jan6"

path = f"csv/{tool}_{sport}/{contest_date}.csv"


def print_current_df(dataframe):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(dataframe)

In [2]:
original_df = pd.read_csv(path)
if tool == "DK":
    original_df[['Game','Date','Time','Zone']] = original_df.loc[:,'Game Info'].str.split(" ", 3, expand = True)
    original_df = original_df.rename(columns={'AvgPointsPerGame':'FPPG','TeamAbbrev':'Team'})
# print_current_df(original_df)
original_df

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,Team,FPPG,Game,Date,Time,Zone
0,C,Nathan MacKinnon (20887611),Nathan MacKinnon,20887611,C/UTIL,9500,WPG@COL 01/06/2022 09:00PM ET,COL,17.27,WPG@COL,01/06/2022,09:00PM,ET
1,G,Darcy Kuemper (20889937),Darcy Kuemper,20889937,G,8700,WPG@COL 01/06/2022 09:00PM ET,COL,13.60,WPG@COL,01/06/2022,09:00PM,ET
2,G,Pavel Francouz (20889938),Pavel Francouz,20889938,G,8700,WPG@COL 01/06/2022 09:00PM ET,COL,7.00,WPG@COL,01/06/2022,09:00PM,ET
3,G,Tristan Jarry (20889939),Tristan Jarry,20889939,G,8600,PIT@PHI 01/06/2022 07:00PM ET,PIT,17.54,PIT@PHI,01/06/2022,07:00PM,ET
4,G,Casey DeSmith (20889940),Casey DeSmith,20889940,G,8600,PIT@PHI 01/06/2022 07:00PM ET,PIT,9.74,PIT@PHI,01/06/2022,07:00PM,ET
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,D,Matt Donovan (20889795),Matt Donovan,20889795,D/UTIL,2500,NSH@LA 01/06/2022 10:30PM ET,NSH,0.00,NSH@LA,01/06/2022,10:30PM,ET
1303,D,Xavier Bouchard (20889797),Xavier Bouchard,20889797,D/UTIL,2500,NSH@LA 01/06/2022 10:30PM ET,NSH,0.00,NSH@LA,01/06/2022,10:30PM,ET
1304,D,Marc Del Gaizo (20889799),Marc Del Gaizo,20889799,D/UTIL,2500,NSH@LA 01/06/2022 10:30PM ET,NSH,0.00,NSH@LA,01/06/2022,10:30PM,ET
1305,D,Matt Tennyson (20889801),Matt Tennyson,20889801,D/UTIL,2500,NSH@LA 01/06/2022 10:30PM ET,NSH,0.00,NSH@LA,01/06/2022,10:30PM,ET


### Modify Original CSV entries to create a DF of relevant data


In [5]:
def convert_positions_to_values(df,original_df):
    
    df.insert(loc=1, column='Pos', value=0)
    conditions = [
        (df['Position'] == 'PG'),
        (df['Position'] == 'SG'),
        (df['Position'] == 'SF'),
        (df['Position'] == 'PF'),
        (df['Position'] == 'C')
    ]
    values = [1, 2, 3, 4, 5]
    df.loc[:,'Pos'] = np.select(conditions, values)

    
def single_game(df,original_df):
    df = original_df[['FPPG','Salary']]
    df.insert(0, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
    
def drop_inactive_players(df):
    df = df[df['Injury Status'] != 'INJ'] 
    df = df[df['Injury Status'] != 'O'] 
    df = df[df['Injury Status'] != 'IR'] 
    return df
def create_df(sport,tool,original_df):
    if tool == "Yahoo":
        df = original_df[['Position', 'Team','Game','Time','Injury Status','FPPG','Salary']].copy()
        df.insert(4, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
        df = drop_inactive_players(df)
        
        if sport == "NHL":
            df.insert(df.columns.get_loc("FPPG"),'Starting',original_df['Starting'])
            df['Value'] = df['FPPG']/df['Salary']
        else:
            number_of_teams = len(original_df['Team'].value_counts())
            if(number_of_teams == 2):
                single_game(df,original_df)
            else:    
                convert_positions_to_values(df,original_df)
                df.head()
    if tool == "DK":
        df = original_df[['Position', 'Name','Team','Game','Time','FPPG','Salary']]
        if sport == "NHL":
            df['Value'] = df['FPPG']/df['Salary']*1000
            df.insert(df.columns.get_loc("FPPG"),'Starting','')
            df.loc[df.Position == 'G','Starting'] = 'No'
    if sport == "NHL":
        df = df.round({'Value': 2})
        df.loc[df['Position'] == 'LW', 'Position'] = 'W'
        df.loc[df['Position'] == 'RW', 'Position'] = 'W'
#     print_current_df(df)
    return df
    
df = create_df(sport,tool,original_df)

### Interface

To be modified as more front end functionality is added

In [6]:
def add_to_lineup(df, player_df, name):
    if player_df['Name'].str.lower().str.contains(name.lower()).any() != True:
        player_df = player_df.append(df.loc[df['Name'].str.lower() == name.lower()])
    else:
        if name != '' : print(name + " is already in list")
    return player_df
def remove_from_lineup(player_df, name):
    print("name? " + name)
    if player_df['Name'].str.lower().str.contains(name.lower()).any() == True:
        player_df.drop(player_df.loc[player_df['Name'].str.lower()==name.lower()].index, inplace=True)
    else:
        if name != ''  : print(name + " is not in in list") 
    return player_df

In [5]:
df2 = df.sort_values(by=['Salary'], ascending = False)
# if sport == 'NHL':
#     df2 = df2.drop(df2[df2.Starting == "No"].index)
df2.drop(df2[df2.FPPG == 0.00].index,inplace=True)
for name in df2['Name']: 
    print(name, end = ", ")


Nathan MacKinnon, Pavel Francouz, Darcy Kuemper, Tristan Jarry, Casey DeSmith, Mikko Rantanen, Max Pacioretty, Justus Annunen, Jeremy Swayman, Linus Ullmark, Robin Lehner, Laurent Brossoit, Kirill Kaprizov, Andrei Vasilevskiy, Brian Elliott, Kevin Lankinen, Logan Thompson, Marc-Andre Fleury, Sidney Crosby, Hugo Alnefelt, Maxime Lagace, John Gibson, Anthony Stolarz, Timo Meier, MacKenzie Blackwood, Jon Gillies, Akira Schmid, Jonathan Bernier, Arvid Soderblom, Collin Delia, James Reimer, Adin Hill, Sergei Bobrovsky, Jonas Johansson, Nico Daws, Brad Marchand, Zach Sawchenko, Juuse Saros, David Rittich, Patrice Bergeron, David Pastrnak, Braden Holtby, Jake Oettinger, Spencer Knight, Kyle Connor, Jonathan Quick, Cal Petersen, Connor Ingram, Cale Makar, Anton Khudobin, Steven Stamkos, Malcolm Subban, Ukko-Pekka Luukkonen, Joonas Korpisalo, Elvis Merzlikins, Jake Guentzel, Jakob Chychrun, Mika Zibanejad, Craig Anderson, Dustin Tokarski, Jacob Markstrom, Dan Vladar, Patrick Kane, Roman Josi, S

### Table Add

In [7]:
def get_names(names):
    return [x.strip() for x in names.split(',')]


##adding these blank columns that can be edited in an excel sheet to add the values
##Future versions will let user change the values inside them
##TODO Modify current excel based version to use excel formulas
##https://stackoverflow.com/questions/51348874/save-pandas-dataframes-with-formulas-to-xlsx-files
def add_extra_columns(player_df):
    player_df['Floor'] = ""
    player_df['Ceiling'] = ""
    if sport == 'NBA':
        player_df['Estimate'] = ""
    player_df['Actual'] = ""
    if sport == 'NBA':
        player_df['Deviation'] = ""
    player_df['Floor Value'] = ""
    player_df['Ceiling Value'] = ""
    player_df['Border Value'] = ""
    
    if sport == 'NBA':
        player_df['Estimate Value'] = ""
        player_df['Actual Value'] = ""
    player_df['Notes'] = ""
    return player_df
def choose_relevant_players(player_df,df):
    
    queries = ['exit', 'all', 'show']
    player_df = player_df.replace(np.nan, '', regex=True)
    
    exit = False
    while exit!=True:
        query = input('Name Of Player:')
        if query not in queries:
            if re.search('^remove', query):
                names_to_remove = re.findall('(?<=remove).*$', query)[0]
                names = get_names(names_to_remove)
                for name in names:
                    name = name.title()
                    player_df = remove_from_lineup(player_df,name)
                    print(f"removed {name}")

            else:
                names = get_names(query)
                for name in names:
                    name = name.title()
                    if df['Name'].str.lower().str.contains(name.lower()).any():
                        player_df = add_to_lineup(df,player_df,name)   
                    else:
                        print(name + " is not in the csv list. Please try again")
        elif query =='show':
            print_current_df(player_df)
        elif query =='all':
            player_df = df
            print_current_df(player_df)
        else:
            print_current_df(player_df)
            exit = True
    return player_df
player_df = pd.DataFrame(columns = df.columns)

player_df = choose_relevant_players(player_df,df)
player_df = add_extra_columns(player_df)


Name Of Player:Nathan MacKinnon, Pavel Francouz, Darcy Kuemper, Tristan Jarry, Casey DeSmith, Mikko Rantanen, Max Pacioretty, Justus Annunen, Jeremy Swayman, Linus Ullmark, Robin Lehner, Laurent Brossoit, Kirill Kaprizov, Andrei Vasilevskiy, Brian Elliott, Kevin Lankinen, Logan Thompson, Marc-Andre Fleury, Sidney Crosby, Hugo Alnefelt, Maxime Lagace, John Gibson, Anthony Stolarz, Timo Meier, MacKenzie Blackwood, Jon Gillies, Akira Schmid, Jonathan Bernier, Arvid Soderblom, Collin Delia, James Reimer, Adin Hill, Sergei Bobrovsky, Jonas Johansson, Nico Daws, Brad Marchand, Zach Sawchenko, Juuse Saros,
Name Of Player:exit


,Position,Name,Team,Game,Time,Starting,FPPG,Salary,Value
0,C,Nathan MacKinnon,COL,WPG@COL,09:00PM,,17.27,9500,1.82
2,G,Pavel Francouz,COL,WPG@COL,09:00PM,No,7.00,8700,0.80
1,G,Darcy Kuemper,COL,WPG@COL,09:00PM,No,13.60,8700,1.56
3,G,Tristan Jarry,PIT,PIT@PHI,07:00PM,No,17.54,8600,2.04
4,G,Casey DeSmith,PIT,PIT@PHI,07:00PM,No,9.74,8600,1.13
5,W,Mikko Rantanen,COL,WPG@COL,09:00PM,,15.56,8600,1.81
10,W,Max Pacioretty,VGK,NYR@VGK,10:00PM,,19.31,8500,2.27
9,G,Justus Annunen,COL,WPG@COL,09:00PM,No,9.65,8500,1.14
7,G,Jeremy Swayman,BOS,MIN@BOS,07:00PM,No,13.83,8500,1.63
6,G,Linus Ullmark,BOS,MIN@BOS,07:00PM,No,15.92,8500,1.87


In [109]:
def output_sheet(output_df):
#     dt = datetime.now()
#     now = dt.strftime("%Y-%m-%d %H_%M")

    file_name = f'output/{tool}/{sport}/{contest_date}_{tool}_{sport}.xlsx'
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
    # header and index and skip one row to allow us to insert a user defined
    # header.

    output_df.to_excel(writer, index = False, header=True)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Get the dimensions of the dataframe.
    (max_row, max_col) = output_df.shape

    # Create a list of column headers, to use in add_table().exo
    column_settings = []
    for header in player_df.columns:
        column_settings.append({'header': header})

    # Add the table.
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

    # Make the columns wider for clarity.
    worksheet.set_column(0, max_col - 1, 12)

    if sport == 'NHL': 
        superstar_df = df.loc[df['Salary'] > 18]
        superstar_df.to_excel (writer, index = False, header=True, sheet_name = "Superstars")
        worksheet2 = writer.sheets['Superstars']

        (max_row, max_col) = df.shape
        print_current_df(superstar_df)
        worksheet2.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

        # Make the columns wider for clarity.
        worksheet2.set_column(0, max_col - 1, 12)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()
    print('succesfully created: ' + file_name)

    
output_sheet(player_df)


succesfully created: output/DK/NBA/jan7_DK_NBA.xlsx


### Single-game analysis [WIP]

#### Single Game Entry Strategy:

- **Stars:** Out of the 4-5 likely star players, choose 3.

- **Peasants:** Out of the 5-10 non-star players, choose 5.



In [10]:
single_df = player_df[['Name','Salary','Floor','Ceiling','Estimate']]
print_current_df(single_df)
create_lineup(single_df)

,Name,Salary,Floor,Ceiling,Estimate
54,Jayson Tatum,46,,,
29,Ja Morant,40,,,
52,Jaylen Brown,37,,,
70,Julius Randle,36,,,
145,Devin Booker,34,,,
104,Brandon Ingram,33,,,
139,Chris Paul,33,,,
101,Jonas Valanciunas,30,,,
13,Saddiq Bey,29,,,
16,Cade Cunningham,29,,,


NameError: name 'create_lineup' is not defined

In [11]:
max_salary = 30

In [12]:
## best 5 probable, best 5 worst, best 5 best. Create combinations of lineups with the most recurring names

def create_lineup(df):
    lineup = []
    df2 = df.ge()
        
    
    print(highest_paid_player.Name)
    

In [ ]:
numbers = [1, 2, 3, 7, 7, 9, 10]
result = [seq for i in range(len(numbers), 0, -1) for seq in itertools.combinations(numbers, i) if sum(seq) >= 10]
print(result)

In [ ]:
class Lineup: 
    superstar = ""
    megastar = ""
    star = ""
    other_players = []   
    

In [ ]:
class MyClass:
    """A simple example class"""
    i = 12345

    def __init__(self,data):
        self.data = data
l = [1,2,"e"]
x = MyClass(l)
x.data


### Multi-game analysis [WIP]

In [ ]:
# sorted_df = filled_df.sort_values(by=['Pos','Probable'],ascending=[True,False]).drop(['Notes', 'Pts', 'Reb', 'Ast', 'Stl', 'Blk', 'TO',
#        'Total'],axis = 1)

# # sorted_df.insert(12,'Worst Value',sorted_df["Worst"] / sorted_df["Salary"])
# # sorted_df.insert(13,'Best Value',sorted_df["Best"] / sorted_df["Salary"])
# df=sorted_df
# lineups = []
# positions = []



# df = df.loc[:, lambda df: ['Pos','Salary','Name', "Worst","Best",'Probable']]
# [positions.append(df.loc[(sorted_df['Pos'] == i)].sort_values(by=['Pos','Probable'],ascending=[True,False])) for i in range(1,6)]

# [print_current_df(position_df) for position_df in positions]

In [ ]:
# def select_eligible(df):
# new_df = sorted_df.loc[(sorted_df['Salary'] > 30)]
# df

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


In [ ]:
# d = pd.concat(
#     [df.loc[a].reset_index(), df.loc[b].reset_index()],
#     keys=['a', 'b'], axis=1
# )

# d
